In [2]:
%autosave

UsageError: %autosave requires an integer, got ''


In [1]:
import tensorflow as tf
from tensorflow import keras

In [9]:
!curl -O https://github.com/alexeygrigorev/mlbookcamp-code/releases/download/chapter7-model/xception_v4_large_08_0.894.h5

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100     9  100     9    0     0     33      0 --:--:-- --:--:-- --:--:--    33


In [4]:
!python -V

Python 3.11.5


In [10]:
from tensorflow.keras.applications.xception import Xception

model = Xception(weights='imagenet', input_shape=(299, 299, 3))

In [11]:
import tensorflow as tf

converter = tf.lite.TFLiteConverter.from_keras_model(model)

tflite_model = converter.convert()

with open('model.tflite', 'wb') as f_out:
    f_out.write(tflite_model)

INFO:tensorflow:Assets written to: /var/folders/53/zdvtpqvs6qxf5ht0lhmgw37c0000gp/T/tmpu2ou09cj/assets


INFO:tensorflow:Assets written to: /var/folders/53/zdvtpqvs6qxf5ht0lhmgw37c0000gp/T/tmpu2ou09cj/assets
2023-09-15 15:40:16.433711: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2023-09-15 15:40:16.433890: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2023-09-15 15:40:16.435069: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /var/folders/53/zdvtpqvs6qxf5ht0lhmgw37c0000gp/T/tmpu2ou09cj
2023-09-15 15:40:16.451762: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-09-15 15:40:16.451779: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /var/folders/53/zdvtpqvs6qxf5ht0lhmgw37c0000gp/T/tmpu2ou09cj
2023-09-15 15:40:16.501911: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:353] MLIR V1 optimization pass is not enabled
2023-09-15 15:40:16.517269: I tensorflow/cc/saved_model/load

In [12]:
import tensorflow.lite as tflite

interpreter = tflite.Interpreter(model_path='model.tflite')
interpreter.allocate_tensors()

INFO: Initialized TensorFlow Lite runtime.
INFO: Applying 1 TensorFlow Lite delegate(s) lazily.
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
VERBOSE: Replacing 104 node(s) with delegate (TfLiteXNNPackDelegate) node, yielding 1 partitions for the whole graph.
INFO: Successfully applied the default TensorFlow Lite delegate indexed at 0.
 *NOTE*: because a delegate has been applied, the precision of computations should be unchanged, but the exact output tensor values may have changed. If such output values are checked in your code, like in your tests etc., please consider increasing error tolerance for the check.


In [13]:
interpreter

In [18]:
!curl -O http://bit.ly/mlbookcamp-pants

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   217  100   217    0     0    503      0 --:--:-- --:--:-- --:--:--   508


In [19]:
from tensorflow.keras.preprocessing.image import load_img

In [25]:
from tensorflow.keras.preprocessing.image import load_img

path = '../deep_learning/clothing-dataset-small/train/t-shirt/'
name = '0a85a584-cb49-4795-b2f1-7eebbf09399a.jpg'
fullname = f'{path}/{name}'
img = load_img(fullname, target_size=(299,299))

In [27]:
from tensorflow.keras.applications.xception import preprocess_input
import numpy as np

In [28]:
x = np.array(img)
X = np.array([x])

X = preprocess_input(X)

In [29]:
X.shape

(1, 299, 299, 3)

In [30]:
preds = model.predict(X)

1/1 [==============================] - 0s 463ms/step


In [31]:
preds

array([[7.79549373e-06, 2.24676969e-05, 1.86893976e-05, 1.20043542e-05,
        3.43274987e-05, 4.67107348e-05, 1.64154571e-05, 6.52924382e-06,
        1.16982710e-05, 8.63576497e-06, 1.62400775e-05, 9.62266222e-06,
        5.37133201e-06, 1.11342770e-05, 7.90367085e-06, 1.57592131e-05,
        1.98552043e-05, 6.96682810e-06, 1.06662283e-05, 1.37064089e-05,
        6.35953984e-05, 7.11989060e-06, 2.08244146e-05, 8.57218947e-06,
        1.69317900e-05, 2.18108144e-05, 9.67478900e-06, 1.47072142e-05,
        2.69732445e-05, 2.08300935e-05, 1.96119290e-05, 1.57811864e-05,
        1.70845196e-05, 3.13309865e-05, 3.48267677e-05, 2.90186545e-05,
        1.64957437e-05, 2.12442719e-05, 2.11680908e-05, 9.63355160e-06,
        5.68349787e-06, 3.21380649e-05, 1.07036349e-05, 1.17406189e-05,
        2.57239662e-05, 3.02084845e-05, 1.00964435e-05, 1.01565802e-05,
        1.47668607e-05, 2.57436477e-05, 3.32666896e-05, 1.37164116e-05,
        2.48509787e-05, 3.22736050e-05, 1.22734637e-05, 1.816216